# FF7R Translation & Localization Script

## Setup and Initialization

In [ ]:
# Import Libraries
import os
import json
import time
import traceback
import re
import pandas as pd
from openai import AzureOpenAI
from datetime import datetime

# Initialize Azure OpenAI client with key-based authentication
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")  
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")
   
client = AzureOpenAI(
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-05-01-preview",  
)

# File to be translated
file_name = "4000-MIDGR_TxtRes"

In [ ]:
# # Test the client
# prompt = "Describe Tifa Lockhart from Final Fantasy VII in a json format"

# completion = client.chat.completions.create(
#     model=deployment,
#     messages=[
#         {"role": "system", "content": "You are an expert in the lore and story of the game Final Fantasy VII which includes the orignal game, remakes and spin-offs."},
#         {"role": "user", "content": prompt}
#     ]
# )

# # Print the response
# print(completion.choices[0].message.content)

# Testing

In [ ]:
def process_dialogue_file(file_path):
    # Load CSV file
    df = pd.read_csv(file_path, encoding="utf-8")
    
    # Remove metadata row where id is 'language'
    df = df[df["id"] != "language"]
    
    # Remove rows where both 'sub_id' and 'text' are empty
    df = df.dropna(subset=["sub_id", "text"], how="all")
    
    # Ensure 'text' column is treated as a string and replace NaN with empty string
    df["text"] = df["text"].fillna("").astype(str)
    
    # Initialize a list to store structured dialogues
    dialogues = []
    
    # Count occurrences of each ID
    id_counts = df["id"].value_counts()
    
    # Iterate through unique IDs
    for unique_id, count in id_counts.items():
        rows = df[df["id"] == unique_id]
        
        if count == 2:
            # If there are two rows, determine speaker and dialogue
            speaker_row = rows[rows["sub_id"] == "ACTOR"]
            dialogue_row = rows[rows["sub_id"].isna()]
            
            if not speaker_row.empty and not dialogue_row.empty:
                speaker = speaker_row.iloc[0]["text"].strip()
                dialogue = dialogue_row.iloc[0]["text"].strip()
                
                if speaker and dialogue:
                    dialogues.append({"id": unique_id, "speaker": speaker, "dialogue": dialogue})
        
        elif count == 1:
            # If there is only one row, assume it's an NPC/system dialogue
            dialogue = rows.iloc[0]["text"].strip()
            if dialogue:
                dialogues.append({"id": unique_id, "speaker": "NPC", "dialogue": dialogue})
    
    # Convert structured dialogues into a DataFrame
    return pd.DataFrame(dialogues)

# Process English and Japanese files
en_file_path = f"./testing/ModifiedExports/{file_name}.csv"
jp_file_path = f"./testing/ModifiedExports/{file_name}_jp.csv"

en_dialogue_df = process_dialogue_file(en_file_path)
jp_dialogue_df = process_dialogue_file(jp_file_path)

# Merge English and Japanese dialogues using a left join
merged_dialogue_df = en_dialogue_df.merge(jp_dialogue_df, on="id", how="left", suffixes=("_en", "_jp"))

# Remove the speaker_jp column since we want to translate dialogue only
merged_dialogue_df = merged_dialogue_df.drop(columns=["speaker_jp"])
merged_dialogue_df.to_csv(f"./testing/ModifiedExports/{file_name}_merged.csv", index=False)

print(f"Relevant dialogues to be translated in {file_name}: {len(merged_dialogue_df)}")

In [ ]:
def clean_json_output(ai_response):
    # If the response contains ```json, extract the content within
    match = re.search(r"```json\s*(.*?)\s*```", ai_response, re.DOTALL)
    if match:
        return match.group(1)  # Extract only the JSON part
    return ai_response  # Return as-is if no backticks are found

In [ ]:
# Read the merged dialogue CSV file
merged_file_path = f"./testing/ModifiedExports/{file_name}_merged.csv"
merged_df = pd.read_csv(merged_file_path, encoding="utf-8")

# Sort by ID
merged_df = merged_df.sort_values("id")

# Extract unique scene identifiers
scene_ids = sorted(set(merged_df["id"].str.split("_").apply(lambda x: "_".join(x[:-3]))))

# Store translations for all scenes
all_translations = []

for scene_id in scene_ids:
    try:
        scene_df = merged_df[merged_df["id"].str.startswith(scene_id)].copy()
        
        # Create the prompt for AI translation
        prompt = f"""
        You are an expert translator specializing in Japanese-to-English localization for video games. Your task is to provide a faithful translation of the following dialogue from Final Fantasy VII Rebirth in a valid JSON format.
        
        Please translate while maintaining:
        - The original tone and context.
        - Character personality and speech style.
        - Natural English phrasing.
        
        Return the results in a structured JSON format with the following structure:
        {{
            "scene_id": "{scene_id}",
            "translations": [
                {{"id": "<original_id>", "translation": "<your improved English translation>"}},
                ...
            ]
        }}
        
        Here is the Japanese dialogue along with its official English localization which you can refer for additional context:
        """
        
        for _, row in scene_df.iterrows():
            prompt += f"\nID: {row['id']}"
            prompt += f"\n{row['speaker_en']} (JP): {row['dialogue_jp']}"
            prompt += f"\n{row['speaker_en']} (EN): {row['dialogue_en']}\n"
        
        prompt += "\nPlease provide only the JSON output formatted as specified." 
        
        # Ping the client
        completion = client.chat.completions.create(
            model=deployment,
            messages=[
                {"role": "system", "content": "You are an expert translator specializing in Final Fantasy VII localization."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=4096
        )
        
        # Parse the JSON response safely
        ai_response = completion.choices[0].message.content
        ai_response_clean = clean_json_output(ai_response)
        translated_data = json.loads(ai_response_clean)
        all_translations.append(translated_data)

        # Save progress after each scene
        with open(f"./testing/translations_backup_{file_name}.json", "w", encoding="utf-8") as f:
            json.dump(all_translations, f, ensure_ascii=False, indent=4)

        # Small delay to avoid hitting rate limits
        time.sleep(2)

        # Log progress
        print(f"Completed scene {scene_id}")

    except Exception as e:
        # Capture error details
        time_now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

        error_details = {
            "timestamp": time_now,
            "scene_id": scene_id,
            "error_message": str(e),
            "traceback": traceback.format_exc(),
            "prompt_used": prompt,
            "response_received": ai_response
        }

        error_log_file = f"./testing/logs/error_{file_name}_{scene_id}.json"

        # Save error details to a file
        with open(error_log_file, "w", encoding="utf-8") as f:
            json.dump(error_details, f, ensure_ascii=False, indent=4)

        print(f"Error processing scene {scene_id}. Logged details to {error_log_file}")

# Convert all translations to DataFrame
final_translation_df = pd.DataFrame([t for scene in all_translations for t in scene["translations"]])

# Save the final translations to a CSV file
final_translation_df.to_csv(f"./testing/ModifiedExports/{file_name}_translated.csv", index=False, encoding="utf-8")

In [ ]:
# Load the original and translated CSV files
df_original = pd.read_csv(f"./testing/ModifiedExports/{file_name}.csv")
df_translated = pd.read_csv(f"./testing/ModifiedExports/{file_name}_translated.csv")

# Merge original and translated datasets on 'id'
df_merged = df_original.merge(df_translated, on="id", how="left")

# Identify dialogue rows that have a matching translation (sub_id is NaN, text is not blank, and translation is not blank)
dialogue_mask = df_merged["sub_id"].isna() & df_merged["text"].notna() & df_merged["translation"].notna()

# Replace text column only for matching instances with a translation
df_merged.loc[dialogue_mask, "text"] = df_merged.loc[dialogue_mask, "translation"]

# Drop the extra 'translation' column after updating
df_merged.drop(columns=["translation"], inplace=True)

# Save the updated file
df_merged.to_csv(f"./testing/ModifiedExports/{file_name}_updated.csv", index=False)